In [1]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import pyarrow.feather as feather
import os

In [ ]:
all_paintings = []
for i in range(31):
    r = requests.get(f'https://www.twoinchbrush.com/season/{i+1}')
    soup = BeautifulSoup(r.content)
    urls = soup.find_all('a')
    urls = [a.get('href') for a in urls if '/painting/' in a.get('href')]
    all_paintings.extend(urls)

In [ ]:
len(all_paintings)

In [ ]:
all_paintings[0]

In [ ]:
results = []

for i, painting in enumerate(all_paintings):
    r = requests.get(f'https://www.twoinchbrush.com{painting}')
    
    soup = BeautifulSoup(r.content)
    title = r.url.replace('https://www.twoinchbrush.com/painting/','')
    imgs = soup.find_all('img')
    img = [img.get('src') for img in imgs if '/images/painting' in img.get('src')]
    img = 'https://www.twoinchbrush.com{}'.format(img[0].replace('webp','png'))
    results.append({'title': title,
                   'link':img,
                   'bob':1})
    divs = soup.find_all('div')
    fans = ['https://www.twoinchbrush.com{}'.format(div.get('data-img')) for div in divs if div.get('data-img') != None]
    for fan in fans:
        results.append({'title': title,
                   'link':fan,
                   'bob':0})
    print(f'{i+1} / {len(all_paintings)}', end='\r')

In [ ]:
episode_links = []
for i in range(4):
    url = f'https://www.bbc.co.uk/programmes/m000hjkz/episodes/player?page={i}'
    r = requests.gBeautifulSouprl)
    soup = BeautifulSoup(r.content)
    links = soup.find_all('h2', class_='programme__titles')
    links = [link.find('a').get('href') for link in liepisode_links]
    episode_links.extend(linksepisode_linksrequests

In [ ]:
for link in episode_links:
    r = requests.get(link)
    soup = BeautifulSoup(r.content)
    div = soup.find('div', class_='episode-playout')
    img = div.find('img').get('src')
    title = soup.find('div', class_='island')
    title = '-'.join(title.find('h1').text.lower().split())
    results.append({'title': title,
                   'link':img,
                   'bob':1})

In [ ]:
pbs_links = []
for i in range(34,39):
    url = f'https://www.pbs.org/show/best-joy-painting/episodes/season/{i}/'
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    soup = soup.find(class_='video-catalog__items')
    pics = soup.find_all(class_='video-catalog__item')
    for pic in pics:
        img = pic.find('source').get('srcset').split()[-2].split('?')[0]
        title ='-'.join(pic.find('h3').text.lower().split())
        results.append({'title': title,
                   'link':img,
                   'bob':1})


In [ ]:
import pandas as pd

df = pd.DataFrame(results)

In [ ]:
feather.write_feather(df, 'data/bob_ross.feather')

In [2]:
df = feather.read_feather('data/bob_ross.feather')

In [14]:
class ImageDownloader:
    
    def __init__(self, dataframe, folder_string):
        self.df = dataframe

        self.folder_string = folder_string
        
        self.path = f'./{folder_string}'
        
        self.bob_dir = self.path+'/bob'
        self.not_bob_dir = self.path+'/not_bob'
        
        self._to_download = len(self.df)
        
    def download(self):
        
        self.downloaded_ = 0
        
        try:
            print(f'created {self.path}')
        except:
            print(f'can\'t create {self.path}')
        try:
            os.mkdir(self.bob_dir)
            print(f'created {self.bob_dir}')
        except:
            print(f'can\'t create {self.bob_dir}')
        try:
            os.mkdir(self.not_bob_dir)
            print(f'created {self.not_bob_dir}')
        except:
            print(f'can\'t create {self.not_bob_dir}')
        

        max_threads = 20
        threads = min(max_threads, len(self.df))

        indexes = self.df.index
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
            executor.map(self._download_imgs_for_set, indexes)
            
    def _download_imgs_for_set(self, index):
        row = self.df.iloc[index]
        filetype = row.link.split('.')[-1]
        filename = f'{row.title}_{index}.{filetype}'
        if row.bob == 1:
            bob_not_bob = 'bob'
        else:
            bob_not_bob = 'not_bob'
            
        path = f'{self.folder_string}/{bob_not_bob}/{filename}'

        img_data = requests.get(row.link)
        
        if img_data.ok:
            with open(path, 'wb') as handler:
                handler.write(img_data.content)
            self.downloaded_ += 1
        else:
            self.bad_links.append(index)
            
        print(f'{self.downloaded_} / {len(self.df)}', end='\r')

In [16]:
try:
    os.mkdir('./data/images')
    os.mkdir('./data/images/all_images')
except:
    pass
downloader = ImageDownloader(df, 'data/images/all_images').download()


created ./data/images/all_images
can't create ./data/images/all_images/bob
can't create ./data/images/all_images/not_bob
